<a href="https://colab.research.google.com/github/paulowoicho/msc_project/blob/master/Summarization_Task_Baselines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Prepare the Data

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
project_id = 'test-281700'
!gcloud config set project {project_id}
!gsutil ls

Updated property [core/project].
gs://spotify_asr_dataset/
gs://staging.test-281700.appspot.com/
gs://test-281700.appspot.com/


In [ ]:
bucket_name = 'spotify_asr_dataset'
#download dataset
!gsutil -m cp -r gs://{bucket_name}/dataset.csv /content/

Copying gs://spotify_asr_dataset/dataset.csv...
/ [1/1 files][  2.9 GiB/  2.9 GiB] 100% Done  53.9 MiB/s ETA 00:00:00           
Operation completed over 1 objects/2.9 GiB.                                      


In [ ]:
import pandas as pd
dataset = pd.read_csv('dataset.csv')

In [ ]:
dataset.head(5)

,episode_id,transcript
0,spotify:episode:399kdfMnjw0KYANZU7CQJ0,It's the mother back a podcast. Well that was...
1,spotify:episode:49wcMBeJfaaL6KFFdsWvac,If you haven't heard about anchor is the easi...
2,spotify:episode:0JOymLFsRdeBVZbEA72ayj,Hello and welcome to the podcast the first ev...
3,spotify:episode:7sHyO8wLeEd1LuxfS8AIls,"Hey, hey. Hey. Hey. Hey, this is your girl Je..."
4,spotify:episode:1WosITIkpJemzZaPh8zAVb,This is the planetary potential podcast for t...


In [ ]:
len(dataset)

105360

In [ ]:
#download gold summaries
!gsutil -m cp -r gs://{bucket_name}/150gold.tsv /content/

#download metadata for episodes
!gsutil -m cp -r gs://{bucket_name}/metadata.tsv /content/

Copying gs://spotify_asr_dataset/150gold.tsv...
/ [1/1 files][379.9 KiB/379.9 KiB] 100% Done                                    
Operation completed over 1 objects/379.9 KiB.                                    
Copying gs://spotify_asr_dataset/metadata.tsv...
\ [1/1 files][112.2 MiB/112.2 MiB] 100% Done                                    
Operation completed over 1 objects/112.2 MiB.                                    


In [ ]:
gold_summaries = pd.read_csv('150gold.tsv', sep='\t')
podcasts_metadata = pd.read_csv('metadata.tsv', sep='\t')

In [ ]:
gold_summaries.head(5)

,show name,episode name,episode id,creator description,EGFB,lexrank summary,EGFB.1,textrank summary,EGFB.2,lsa summary,EGFB.3,quasi-supervised summary,EGFB.4,supervised summary,EGFB.5
0,Alpha Male Strategies,Passive Aggressive Women & Developing Mental S...,spotify:episode:4KRC1TZ28FavN3J5zLHEtQ,Boost the podcast! Leave a 5-star review on th...,B,All right guys now as y'all guys might know so...,G,There's no such thing as talk about passengers...,F,I'll pay for all you guys who don't know what ...,F,All women a passive-aggressive. When a woman w...,G,Rejection is a woman’s way of saying you’re no...,B
1,I Hope the Day Has a Good YOU!,"If You Are Bored, You Are Boring.",spotify:episode:4tdDQcsBOUVWnA9XrpgTzS,☀️ aGoodYOU.com 🧡 Discuss this episode in 🗣: f...,B,It was the first and last time I ever said tha...,E,The answer usually comes in the form of a whis...,E,It was the first and last time I ever said tha...,F,If you are bored you are boring. Most people m...,E,"If you are bored, you are boring. --- Suppo...",B
2,American English Grammar Review,Prepositions of Movement Review Two American E...,spotify:episode:626YAxomH0HZ6nCW9NLlGY,Prepositions of movement review two is the sec...,F,So off is again the opposite of on what about ...,B,So off is again the opposite of on what about ...,B,Let's start out with into and out of into is t...,B,Prepositions of movement in this lesson. Let's...,B,"In, out, and off refer to services differently...",B
3,Simmers Digest,Simmer's Digest ep. 1.0 : Introductions,spotify:episode:6AUFl7KQWN6pzGFEIEKFQu,Welcome one and all to the newest the first ev...,F,I hope you enjoy it and I am excited to get to...,F,My passion for The Sims 4 Grew From consuming ...,E,So sit back turn your volume up to 11 and let'...,B,Techobabble is the host of the simmers digest ...,F,Welcome to the very first episode of The Simme...,B
4,Kitty's Pod,TXT Run Away Reaction,spotify:episode:6C4V9iKa9qygtvJCngPO93,TXT Run Away Reaction --- This episode is s...,B,And I will tell you that I am very thrilled an...,B,And I will tell you that I am very thrilled an...,B,"But before we get into that, if you haven't he...",B,Anchor is the easiest way to make a podcast. T...,B,Listen to the song Runaway by txt --- This...,B


In [ ]:
podcasts_metadata.head(5)

,show_uri,show_name,show_description,publisher,language,rss_link,episode_uri,episode_name,episode_description,duration,show_filename_prefix,episode_filename_prefix
0,spotify:show:2NYtxEZyYelR6RMKmjfPLB,Kream in your Koffee,A 20-something blunt female takes on the world...,Katie Houle,['en'],https://anchor.fm/s/11b84b68/podcast/rss,spotify:episode:000A9sRBYdVh66csG2qEdj,1: It’s Christmas Time!,On the first ever episode of Kream in your Kof...,12.700133,show_2NYtxEZyYelR6RMKmjfPLB,000A9sRBYdVh66csG2qEdj
1,spotify:show:15iWCbU7QoO23EndPEO6aN,Morning Cup Of Murder,Ever wonder what murder took place on today in...,Morning Cup Of Murder,['en'],https://anchor.fm/s/b07181c/podcast/rss,spotify:episode:000HP8n3hNIfglT2wSI2cA,The Goleta Postal Facility shootings- January ...,"See something, say something. It’s a mantra ma...",6.019383,show_15iWCbU7QoO23EndPEO6aN,000HP8n3hNIfglT2wSI2cA
2,spotify:show:6vZRgUFTYwbAA79UNCADr4,Inside The 18 : A Podcast for Goalkeepers by G...,Inside the 18 is your source for all things Go...,Inside the 18 GK Media,['en'],https://anchor.fm/s/81a072c/podcast/rss,spotify:episode:001UfOruzkA3Bn1SPjcdfa,Ep.36 - Incorporating a Singular Goalkeeping C...,Today’s episode is a sit down Michael and Omar...,43.616333,show_6vZRgUFTYwbAA79UNCADr4,001UfOruzkA3Bn1SPjcdfa
3,spotify:show:5BvKEjaMSuvUsGROGi2S7s,Arrowhead Live!,Your favorite podcast for everything @Chiefs! ...,Arrowhead Live!,['en-US'],https://anchor.fm/s/917dba4/podcast/rss,spotify:episode:001i89SvIQgDuuyC53hfBm,Episode 1: Arrowhead Live! Debut,Join us as we take a look at all current Chief...,58.189200,show_5BvKEjaMSuvUsGROGi2S7s,001i89SvIQgDuuyC53hfBm
4,spotify:show:7w3h3umpH74veEJcbE6xf4,FBoL,"The comedy podcast about toxic characters, wri...",Emily Edwards,['en'],https://www.fuckboisoflit.com/episodes?format=rss,spotify:episode:0025RWNwe2lnp6HcnfzwzG,"The Lion, The Witch, And The Wardrobe - Ashley...",The modern morality tail of how to stay good f...,51.782050,show_7w3h3umpH74veEJcbE6xf4,0025RWNwe2lnp6HcnfzwzG


In [ ]:
#one show can span multiple episodes
podcasts_metadata[podcasts_metadata['show_uri'] == 'spotify:show:2NYtxEZyYelR6RMKmjfPLB']

,show_uri,show_name,show_description,publisher,language,rss_link,episode_uri,episode_name,episode_description,duration,show_filename_prefix,episode_filename_prefix
0,spotify:show:2NYtxEZyYelR6RMKmjfPLB,Kream in your Koffee,A 20-something blunt female takes on the world...,Katie Houle,['en'],https://anchor.fm/s/11b84b68/podcast/rss,spotify:episode:000A9sRBYdVh66csG2qEdj,1: It’s Christmas Time!,On the first ever episode of Kream in your Kof...,12.700133,show_2NYtxEZyYelR6RMKmjfPLB,000A9sRBYdVh66csG2qEdj
11957,spotify:show:2NYtxEZyYelR6RMKmjfPLB,Kream in your Koffee,A 20-something blunt female takes on the world...,Katie Houle,['en'],https://anchor.fm/s/11b84b68/podcast/rss,spotify:episode:0sTNg31EACSHfZlt41RHmS,2: Tan Hands Save Lives,The do’s & don’ts of self-tanning. A weird con...,20.016367,show_2NYtxEZyYelR6RMKmjfPLB,0sTNg31EACSHfZlt41RHmS
46132,spotify:show:2NYtxEZyYelR6RMKmjfPLB,Kream in your Koffee,A 20-something blunt female takes on the world...,Katie Houle,['en'],https://anchor.fm/s/11b84b68/podcast/rss,spotify:episode:3Ny7dKZ1QHZwadslXJ8Umf,6: #BYOD (with Liz Pickles),On this week’s episode the gorgeous Liz Pickle...,80.472617,show_2NYtxEZyYelR6RMKmjfPLB,3Ny7dKZ1QHZwadslXJ8Umf


In [ ]:
full_dataset = pd.merge(left=podcasts_metadata, right=dataset, how='left', left_on='episode_uri', right_on='episode_id')
del full_dataset['episode_uri']

In [ ]:
full_dataset.head(5)

,show_uri,show_name,show_description,publisher,language,rss_link,episode_name,episode_description,duration,show_filename_prefix,episode_filename_prefix,episode_id,transcript
0,spotify:show:2NYtxEZyYelR6RMKmjfPLB,Kream in your Koffee,A 20-something blunt female takes on the world...,Katie Houle,['en'],https://anchor.fm/s/11b84b68/podcast/rss,1: It’s Christmas Time!,On the first ever episode of Kream in your Kof...,12.700133,show_2NYtxEZyYelR6RMKmjfPLB,000A9sRBYdVh66csG2qEdj,spotify:episode:000A9sRBYdVh66csG2qEdj,Hello. Hello. Hello everyone. This is Katie a...
1,spotify:show:15iWCbU7QoO23EndPEO6aN,Morning Cup Of Murder,Ever wonder what murder took place on today in...,Morning Cup Of Murder,['en'],https://anchor.fm/s/b07181c/podcast/rss,The Goleta Postal Facility shootings- January ...,"See something, say something. It’s a mantra ma...",6.019383,show_15iWCbU7QoO23EndPEO6aN,000HP8n3hNIfglT2wSI2cA,spotify:episode:000HP8n3hNIfglT2wSI2cA,There were two more murders 15 miles away arr...
2,spotify:show:6vZRgUFTYwbAA79UNCADr4,Inside The 18 : A Podcast for Goalkeepers by G...,Inside the 18 is your source for all things Go...,Inside the 18 GK Media,['en'],https://anchor.fm/s/81a072c/podcast/rss,Ep.36 - Incorporating a Singular Goalkeeping C...,Today’s episode is a sit down Michael and Omar...,43.616333,show_6vZRgUFTYwbAA79UNCADr4,001UfOruzkA3Bn1SPjcdfa,spotify:episode:001UfOruzkA3Bn1SPjcdfa,Welcome to inside the 18. Today's episode is ...
3,spotify:show:5BvKEjaMSuvUsGROGi2S7s,Arrowhead Live!,Your favorite podcast for everything @Chiefs! ...,Arrowhead Live!,['en-US'],https://anchor.fm/s/917dba4/podcast/rss,Episode 1: Arrowhead Live! Debut,Join us as we take a look at all current Chief...,58.189200,show_5BvKEjaMSuvUsGROGi2S7s,001i89SvIQgDuuyC53hfBm,spotify:episode:001i89SvIQgDuuyC53hfBm,Hey cheese fans before we get started. I want...
4,spotify:show:7w3h3umpH74veEJcbE6xf4,FBoL,"The comedy podcast about toxic characters, wri...",Emily Edwards,['en'],https://www.fuckboisoflit.com/episodes?format=rss,"The Lion, The Witch, And The Wardrobe - Ashley...",The modern morality tail of how to stay good f...,51.782050,show_7w3h3umpH74veEJcbE6xf4,0025RWNwe2lnp6HcnfzwzG,spotify:episode:0025RWNwe2lnp6HcnfzwzG,"Sorry to interrupt the show, but I do have to..."


#Recreate Baselines (TextRank, LexRank, Lsa)

In [ ]:
one_fifty_gold_summaries = full_dataset.loc[full_dataset["episode_id"].isin(gold_summaries['episode id'])]
len(one_fifty_gold_summaries)

150

In [ ]:
one_fifty_gold_summaries = one_fifty_gold_summaries[['episode_id', 'episode_name', 'episode_description', 'transcript']]
one_fifty_gold_summaries

,episode_id,episode_name,episode_description,transcript
1926,spotify:episode:08hXUWN6aOnHULXrqMiwTi,Recruiting Secrets From a MLM Recruiting Monst...,If you want to start mastering recruiting whic...,Hello everybody. What's going on in this Jess...
2692,spotify:episode:0CExTNH4LFqp1ec1mhTd4I,A Public Service Announcement from the mrbrown...,Don't be a silent victim of crime. a parody fr...,"Ladies, have you been molested don't be a vic..."
2704,spotify:episode:0CGkzBarXCoYA24w5buABS,Favorite parts!!!,We will share some of our favorite parts of Ha...,"Hello everybody, welcome back to butter girls..."
2943,spotify:episode:0DKARHBAz6GNwwIBz9Y4Sk,Fear? I Don't Know Her!,"Hey eaters!! To commemorate spooky season, thi...","Hey heaters, it's Daisy and Abby and we are t..."
3463,spotify:episode:0Fg2g7eNnVBUj9FmtN8uKd,Unschool -001 /Introduction,Introduction | Show Formalities | What to Expe...,"Hey, what's up, this is Uncle Sharma from the..."
...,...,...,...,...
99028,spotify:episode:7JdHnC4Jm05d7gsvpt2iMN,Self Guided 30 Minute Meditation | Meditation ...,Learn to meditate by practicing your own medit...,Thank you for joining me for another piece pr...
99372,spotify:episode:7LBJuMprDt6nN8lWZhw1ZN,The Lady Lemur,"This week, John & Kat are joined by the Lady L...",Welcome to the Leaky nib a podcast about pens...
100490,spotify:episode:7dEbuTgXCXjIufsfgyohnI,1. Pilot,"Bruce Forsyth, Sherlock's hair and time travel...",I've got something very excited to tell you t...
104281,spotify:episode:7uv6JrgV01fiecvCgxRMy8,Best Organic Dessert Recipes,Comfort Foods for Cold Nights,Ingredients bolognese sauce 1 large onion coa...


In [ ]:
!pip install sumy

In [ ]:
!python sumy_example.py

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
This is a sample sentence. My laptop is on my bed. 


In [ ]:
from sumy_example import summarize

one_fifty_gold_summaries['textrank_summary'] = one_fifty_gold_summaries.apply(lambda row: summarize(row['transcript'], 'text_rank'), axis=1)
one_fifty_gold_summaries['lexrank_summary'] = one_fifty_gold_summaries.apply(lambda row: summarize(row['transcript'], 'lex_rank'), axis=1)
one_fifty_gold_summaries['lsa_summary'] = one_fifty_gold_summaries.apply(lambda row: summarize(row['transcript'], 'lsa'), axis=1)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
This is a sample sentence. My laptop is on my bed. 


/usr/local/lib/python3.6/dist-packages/sumy/summarizers/lsa.py:76: UserWarning: Number of words (40) is lower than number of sentences (157). LSA algorithm may not work properly.
  warn(message % (words_count, sentences_count))
/usr/local/lib/python3.6/dist-packages/sumy/summarizers/lsa.py:76: UserWarning: Number of words (1076) is lower than number of sentences (1149). LSA algorithm may not work properly.
  warn(message % (words_count, sentences_count))


In [ ]:
one_fifty_gold_summaries.iloc[3]['textrank_summary']

"Yeah, and it gives you opportunity to be that courageous person where if you weren't afraid of something you would never know that it was actually like something you could have done and then they talked kind of talk about the fight or flight response and it's the eye  Dia of fear keeps us safe in a lot of situations like you and I are both very aware of where we are at night and looking around to different people and making sure we're safe and that fear Keeps Us Alive everyday. Yeah before you can actually show forth, which is one of the hardest parts of overcoming fear because admitting fear to yourself is scary foot and I got anything for your own fear to um, yeah. "

In [ ]:
one_fifty_gold_summaries.iloc[3]['lexrank_summary']

"I kind of get that fear also a side note of what I'm afraid of kidnap being kidnapped is probably the biggest one but in a much more real sense, it's going like I'm not accomplishing enough for giving enough that really ties in with the time thing because it says it yeah, we should be having something done. And what's my fear? "

In [ ]:
one_fifty_gold_summaries.iloc[3]['lsa_summary']

"Yeah, so the idea of habituation  In the psychology today article talks about how it's when our nervous system returns to a more comfortable state, which is a good thing. When you happen to be in a very dangerous situation, once you return to this state of habituation, then you know that your body has calmed down but then we don't go outside and explore to a mode and we can go on autopilot and then then we're not as aware that we're doing it because we're just naturally doing it. "

In [ ]:
one_fifty_gold_summaries.iloc[3]['episode_description']

"Hey eaters!! To commemorate spooky season, this week's podcast will make you reevaluate your deepest darkest fears! "